In [1]:
!pip install requests

In [2]:
import pandas as pd
import numpy as np
import requests

# Начало

Первым делом рассмотрим, какие данные были получены на вход

In [3]:
dataset = pd.read_csv('./2024_400k.csv')
dataset.head()

,link,description
0,https://cdn-st.rutubelist.ru/media/b0/e9/ef285...,"#нарезкистримов , #dota2 , #cs2 , #fifa23 , #m..."
1,https://cdn-st.rutubelist.ru/media/39/6c/b31bc...,🤫НЕ ВВОДИ ЭТУ КОМАНДУ В РОБЛОКС ! #shorts #rob...
2,https://cdn-st.rutubelist.ru/media/e9/e0/b47a9...,"#boobs , #красивыедевушки , #ass"
3,https://cdn-st.rutubelist.ru/media/87/43/b11df...,NaN
4,https://cdn-st.rutubelist.ru/media/d1/e7/642dc...,NaN


In [4]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400000 entries, 0 to 399999
Data columns (total 2 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   link         400000 non-null  object
 1   description  345081 non-null  object
dtypes: object(2)
memory usage: 6.1+ MB


In [5]:
rows_count = dataset['link'].count()
na_rows_count = dataset[dataset['description'].isna()]['link'].count();

print("Общее количество записей {}".format(rows_count))
print("Количество записей без тегов {}".format(na_rows_count))
print("Процент записей без тегов {:.0%}".format(na_rows_count/rows_count))

Общее количество записей 400000
Количество записей без тегов 54919
Процент записей без тегов 14%


После первой попытки обобщенного анализа имеем:
- Датасет состоит из 400 000 записей
- Одна запись содержит линку на видео, и тэги, актуальные для нее
- Имеем, что ~55 000 записей (14%) не имеют тегов, в принципе

Попробуем понять, какой объем данных мы имеем. Это важно для того, чтобы выработать стратегию обучения. Мы знаем, что сервис организатора представляет собой платформу для просмотра коротких видео. Следовательно, можно предположить, что размер контента +- одинаковый. То есть, для примерного подсчета размера выборки будем придерживаться следующей логики:
- С помощью get запроса по рандомно взятому link получим видео и взглянем на размер ответа в header'ах
- Полученный размер возьмем за среднее. Не будем заниматься тем, что реально посчитаем средний размер по выборке или же полностью просканим 400к записей для того, чтобы получить ответ. Выглядит излишним и шумным. На конечный результат, судя по вводной относительно логики работы платформы, данная точность повлияет с небольшим размахом, который в реальности не имеет смысла.
- Умножим средний размер на количество записей, зафиксируем полученный результат
- Порассуждаем над стратегией работы с данными

In [6]:
sizes = []
for i in range(0, 10):
    test_link = dataset['link'][i]
    response = requests.get(test_link)
    size = int(response.headers['Content-Length']) / 1024 / 1024
    sizes.append(size)
    
meanSize = np.array(sizes).mean()

print(f"Средний размер файла {meanSize:.2f} мб")

meanVolume = meanSize * 400000 / 1024 / 1024


print(f"Средний размер выборки {meanVolume:.2f} тб")

Средний размер файла 6.09 мб
Средний размер выборки 2.32 тб


Итак, имеем, что прогнозируемый размер выборки - 2.32тб. Объем достаточно внушительный для того, чтобы работать с данными дефолтным образом и локально. Ощущение, что обработку видео придется осуществлять батчами (пачками). Также необходимо учесть, что само видео непосредственно получаем GET'ом с хоста, который предоставили организаторы. И здесь нужно отметить, что:
- можем падать по rate limiter'у, из-за того, что он наверняка есть)))
- хост может быть перегружен, наверняка команды соперников так же будут к нему обращаться
- возможны задержки в отдаче в пиковые моменты, надо подумать над увеличением таймаутов 
- подумать о ретраях и вообще обработке exception'ов от хоста

И прежде чем перейти к обработке взглянем на то, а какое количество видео имеет идентичные хэштеги

In [7]:
unique_values = dataset['description'].unique()
unique_values_percent = unique_values.size/ dataset['description'].size
print('Количество уникальных описаний в виде хештегов к видео:',unique_values.size)
print('Процент уникальных описаний в виде хештегов к видео:',  f"{unique_values_percent:.0%}")

Количество уникальных описаний в виде хештегов к видео: 91075
Процент уникальных описаний в виде хештегов к видео: 23%


Данная статистика навела и сессия ответов на вопросы навела на мысль о том, что в выборке могут быть дубли по контенту. Очевидно, что вряд ли корректным равенством является видео 1 = видео 2, если равны хэштеги, но, на всякий случай, было решено, что:
- будем смотреть на 1 фрейм видео
- считать хеш от него, и проверять, есть ли в кэше результаты работы моделей по видео. 
- В случае, если хеши совпали, будем пропускать обработку данного видео и обогощать его из кэша

Теперь перейдем к описанию стратегии по обработке

# Стратегия обработки видео

Начнем от простого к сложному, поэтому распишу максимально абстрактно. Предпологаем, что будем действовать следующим образом:
- Брать n видео из выборки
- Смотрим, есть ли описание по видео в кэше, если нет, то следуем дальше по списку вниз
- Полученные видео прогоняем через whisper, в надежде получить транскрибацию с видео
- Сэмплируем каждое видео на m картинок
- Строим архитектуру под Image Captioning. Предварительно будем брать архитектуры, которые обучены на Flickr или COCO 2017ого года, и файнтюнить их, в идеале брать обученные
- Перед тем, как начать основной флоу, возьмем первый кадр с видео и глянем на его хеш
- В случае, если данный хеш обратывался ранее - возьмем 
- Прогоняем полученные m картинок, получаем результат, переводим его и объединяем все в одну строку
- Полученные описания (текст с whisper, текст с описания изображения) складываем в БД. Предварительно будем использовать Elastic. Также открытый вопрос


Дальнейшие действия будем осуществлять на Kaggle, так как у каждого разработчика MAC OS и для использования GPU нужны дополнительные команды. Тестово начнем удаленно, посмотрим на скорость работы заданного алгоритма. Если все будет плохо и обработка будет занимать длительное время, перепишем код под локальное железо и продолжим на MAC OS.

Далее смотреть тетрадку **preprocess_description_with_examples**